In [ ]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [ ]:
if iskaggle:
    pip install datasets --find-links /kaggle/input/us-patent-phrase-to-phrase-matching/frozen_packages --no-index

In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
import random
import os
import torch
from sklearn.model_selection import KFold, StratifiedKFold
import shutil
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoConfig, AutoTokenizer, get_linear_schedule_with_warmup, TrainingArguments, Trainer, AutoModelForSequenceClassification
from datasets import load_metric
import datasets
from transformers import BertModel
import torch.nn as nn
from torch.optim import Adam
from tqdm import tqdm
import warnings, transformers, logging, torch

In [ ]:
os.environ["WANDB_DISABLED"] = "true"
warnings.simplefilter('ignore')
logging.disable(logging.WARNING)

In [ ]:
class CFG:
    if iskaggle:
        input_path = '../input/us-patent-phrase-to-phrase-matching'
    else:
        input_path = '/home/bhavik/projects/kaggle-patent-phrase-matching/data'
    model_path = 'anferico/bert-for-patents'

In [ ]:
if iskaggle:
    titles = pd.read_csv(f"../input/us-patents-category-titles/titles.csv")
else:
    titles = pd.read_csv(f"{CFG.input_path}/titles.csv")

In [ ]:
def prep_input_tokens(df):
    return df['title'] + ' ' + df['anchor']

In [ ]:
test_df = pd.read_csv(f"{CFG.input_path}/test.csv")
test_df = test_df.merge(titles, left_on='context', right_on='code')
test_df['input'] = prep_input_tokens(test_df)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('uspppm_0')

class InferDataset(Dataset):
    def __init__(self, df):
        self.inputs = df['input'].values.astype(str)
        self.targets = df['target'].values.astype(str)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, item):
        inputs = self.inputs[item]
        targets = self.targets[item]
        
        return {
        **tokenizer( inputs, targets )
    }

In [ ]:

test_predictions = []

for fold in range(CFG.num_fold):

    test_dataset = InferDataset(test_df)
    test_outputs = trainer.predict(test_dataset)
    test_prediction = test_outputs.predictions.reshape(-1)
    test_predictions.append(test_prediction)


In [ ]:
test_predictions = np.mean(test_predictions, axis=0)
submission = datasets.Dataset.from_dict({
    'id': test_df['id'],
    'score': test_predictions,
})

submission.to_csv('submission.csv', index=False)